# Train a customer RNN for time series forecasting

## Import libraries

In [48]:
import numpy
import pandas as pd
import math
import random
import keras
from keras.models import Sequential
from keras.models import load_model, Model
from keras.layers import Dense, Input, GRU, concatenate, Activation
from keras.optimizers import Adam
from keras.layers.core import Dense, Dropout, Activation
from collections import deque
from pprint import pprint
import csv
import h5py
import os

#from customRNN import trainRNN


In [49]:
def collect_data_sets(datapath='./src/datasets'):
    """
        Assumes the files are in the following format:
        XXX_train.csv, XXXX_test.csv
        Given a datapath, iterate through the 
        directory, collecting the files and returning a dict
        containing the files and paths to their training
        and test files.
    """
    file_collection = list()
    asset_names = set()
    file_dict = dict()
    
    for filename in os.listdir(datapath):
        # Directory will contain both the npy and csv files
        # The format is XXXX_train.csv or XXXX_test.csv
        if filename.endswith(".csv") and filename.find("_") is not -1:
                full_data_path = os.path.join(datapath, filename)
                assetname = filename[:filename.index("_")]
                asset_names.add(assetname)
                file_collection.append(full_data_path)
    
    for asset in asset_names:
        # create a dict where key is asset name and value is dict of train, test paths
        asset_files = [i for i in file_collection if asset in i]
        temp_dict = dict()
        for file in asset_files:
            if file.find("test") is not -1:
                temp_dict["test"] = file
            if file.find("train") is not -1:
                temp_dict["train"] = file
        file_dict[asset] = temp_dict
                
    
    return file_dict

## Define Hyperparameters

In [50]:
lag = 10
horiz = 10
nfeatures = 1
nunits = 64            # Number of GRUs in recurrent layer
nepochs = 100         # Number of epochs (originally 1000)
d = 0.2                # Percent of neurons to drop at each epoch
optimizer = 'adam'     # Optimization algorithm (also tried rmsprop)
activ = 'elu'          # Activation function for neurons (elu faster than sigmoid)
activr = 'hard_sigmoid'# Activation function for recurrent layer
activd = 'linear'      # Dense layer's activation function
lossmetric = 'mean_absolute_error'  # Loss function for gradient descent
verbose = True        # Whether or not to list results of each epoch

## Import Data and Train

In [61]:
def perform_RNN_training(asset_name, train_datapath, test_datapath):
    
    # csv will be the .train csv
    df = pd.read_csv(train_datapath)

    # Prepare data
    df["Adjclose"] = df.Close # Moving close price to last column
    df.drop(['Date','Close','Adj Close'], 1, inplace=True)
    #df = df.diff() 

    data = df.as_matrix()
    lags = []
    horizons = []
    nsample = len(data) - lag - horiz  # Number of time series (Number of sample in 3D)

    for i in range(nsample):
                        lags.append(data[i: i + lag , -nfeatures:])
                        horizons.append(data[i + lag : i + lag + horiz, -1])

    lags = numpy.array(lags)
    horizons = numpy.array(horizons)
    print("Number of horizons: ", len(horizons))
    lags = numpy.reshape(lags, (lags.shape[0], lags.shape[1], nfeatures))


    # Design RNN architecture
    rnn_in = Input(shape = (lag, nfeatures), dtype = 'float32', name = 'rnn_in')
    rnn_gru = GRU(units = nunits, return_sequences = False, activation = activ, recurrent_activation = activr, dropout = d)(rnn_in)
    rnn_out = Dense(horiz, activation = activd, name = 'rnn_out')(rnn_gru)
    model = Model(inputs = [rnn_in], outputs = [rnn_out])
    model.compile(optimizer = optimizer, loss = lossmetric)

    # Train model
    fcst = model.fit({'rnn_in': lags},{'rnn_out': horizons}, epochs=nepochs,verbose=verbose)

    # Save model
    model_str = './src/{}-custom-rnn.h5'.format(asset_name)
    model.save(model_str)
  
    # Prepare static set of forecasts for experimentation purpose
    horizonpath = './src/datasets/{}_test_horizons.npy'.format(asset_name)
    # Read the asset price time series 
    t = 0
    vec = []
    payload = []
    lines = open(test_datapath, "r").read().splitlines()


    for line in lines[1:]:
        vec.append(float(line.split(",")[4]))
        _d = t - lag + 1
        block = vec[_d:t + 1] if _d >= 0 else -_d * [vec[0]] + vec[0:t + 1] # pad with t0
        payload.append(block)
        t += 1

    payload = numpy.array(payload)
    payload = numpy.reshape(payload, (payload.shape[0], payload.shape[1], 1))
    print('Payload array shape: {}'.format(payload.shape))
    _horiz = model.predict(payload)
    print('Forecasted horizons array shape: {}'.format(_horiz.shape))
    print('Saved at '.format(horizonpath))
    numpy.save(horizonpath, numpy.array(_horiz)) # fmt="%s")

In [62]:
data_sets = collect_data_sets()

# Perform RNN Training and generate static forecasts
for asset, file_dict in results.items():
    # train each
    print('{} train {}, test {}'.format(asset, file_dict['train'], file_dict['test']))
    perform_RNN_training(asset, file_dict['train'], file_dict['test'])


MSFT train ./src/datasets/MSFT_train.csv, test ./src/datasets/MSFT_test.csv
Number of horizons:  1743
Epoch 1/100
 288/1743 [===>..........................] - ETA: 0s - loss: 28.8115

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/module/bucketing_module.py:426: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.03125). Is this intended?
  force_init=force_init)


1743/1743 [==============================] - 1s 396us/step - loss: 15.6681
Epoch 2/100
1743/1743 [==============================] - 1s 367us/step - loss: 1.9274
Epoch 3/100
1743/1743 [==============================] - 1s 380us/step - loss: 1.0094
Epoch 4/100
1743/1743 [==============================] - 1s 366us/step - loss: 1.0091
Epoch 5/100
1743/1743 [==============================] - 1s 366us/step - loss: 1.1020
Epoch 6/100
1743/1743 [==============================] - 1s 369us/step - loss: 0.9930
Epoch 7/100
1743/1743 [==============================] - 1s 366us/step - loss: 1.0103
Epoch 8/100
1743/1743 [==============================] - 1s 368us/step - loss: 0.9571
Epoch 9/100
1743/1743 [==============================] - 1s 375us/step - loss: 0.9676
Epoch 10/100
1743/1743 [==============================] - 1s 365us/step - loss: 0.9513
Epoch 11/100
1743/1743 [==============================] - 1s 369us/step - loss: 1.0246
Epoch 12/100
1743/1743 [==============================] - 1s 36

## Annexes / Experiment sandbox

In [63]:
test = numpy.load(horizonpath)

NameError: name 'horizonpath' is not defined

In [6]:
# Function to return sigmoid
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

print(test[0])
block = test[0]
res = []
for i in range(lag - 1):
    res.append(sigmoid(block[i + 1] - block[i]))

print(res)

[529.67676 528.69305 530.6086  529.74426 530.647   527.3389  531.3673
 530.2965  530.93933 529.00964]
[0.2721575242703309, 0.8716388437180185, 0.29643780508940987, 0.7115060819130747, 0.0352962465878903, 0.9825082834813594, 0.25525082755179346, 0.6553911586659448, 0.126785173178859]
